# 🧪 Demo Interativa - ChurnAPI E2E

Este notebook demonstra como conectar, autenticar e interagir com a API Híbrida de Churn (Java + AI Service).

### Requisitos
- API rodando em `localhost:9999`
- `requests`, `pandas`, `matplotlib` (opcional)

In [ ]:
import requests
import pandas as pd
import io
import time
import json

BASE_URL = "http://localhost:9999"

## 1. Autenticação e Login
Obtém o token JWT para realizar chamadas seguras.

In [ ]:
def get_token(usuario="demo_user", senha="123"):
    # Tentar registrar (ignorar se ja existe)
    try:
        requests.post(f"{BASE_URL}/usuarios", json={"login": usuario, "senha": senha})
    except:
        pass
        
    # Login
    resp = requests.post(f"{BASE_URL}/login", json={"login": usuario, "senha": senha})
    if resp.status_code == 200:
        token = resp.json()["token"]
        print("✅ Login realizado. Token obtido.")
        return token
    else:
        print(f"❌ Erro no login: {resp.status_code}")
        return None

token = get_token()

## 2. Testando GraphQL (Single Mutation & Query)
Vamos cadastrar um cliente único e solicitar a análise de Churn.

In [ ]:
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

# Mutation
mutation = """
mutation {
  registrarAnalise(input: {
    clienteId: "NOTEBOOK-CLIENT-001",
    idade: 35,
    genero: "Masculino",
    regiao: "Sudeste",
    valorMensal: 59.90,
    tempoAssinaturaMeses: 12,
    planoAssinatura: "Premium",
    metodoPagamento: "Cartão",
    dispositivoPrincipal: "Mobile",
    visualizacoesMes: 25
  }) {
    id
    clienteId
    probabilidadeChurn
    modeloUsado
  }
}
"""

resp = requests.post(f"{BASE_URL}/graphql", json={"query": mutation}, headers=headers)
print("GraphQL Response:")
print(json.dumps(resp.json(), indent=2))

## 3. Processamento Otimizado em Lote (CSV)
Envia um arquivo CSV e recebe o resultado processado.

In [ ]:
csv_file = "teste_batch.csv"
url_batch = f"{BASE_URL}/api/churn/batch/optimized"

print(f"📤 Enviando {csv_file}...")
start = time.time()

with open(csv_file, 'rb') as f:
    # O endpoint espera Part 'file'
    files = {'file': (csv_file, f, 'text/csv')}
    # Headers não podem ter Content-Type fixo aqui (requests cuida do boundary)
    h = {"Authorization": f"Bearer {token}"}
    
    r = requests.post(url_batch, files=files, headers=h)

end = time.time()

if r.status_code == 200:
    print(f"✅ Sucesso em {end-start:.2f}s")
    # Ler CSV retornado para DataFrame
    df_result = pd.read_csv(io.BytesIO(r.content))
    display(df_result.head())
else:
    print("❌ Falha no batch:", r.text)

### Gráfico de Distribuição de Churn
Se houver dados, vamos visualizar a probabilidade.

In [ ]:
if 'df_result' in locals():
    try:
        df_result['probabilidadeChurn'].hist(bins=20)
    except:
        print("Sem dados para plotar")